# Objects Detection using SSD

### Importing the libraries

In [3]:
import torch
from torch.autograd import Variable
from data import BaseTransform, VOC_CLASSES as labelmap
from ssd import build_ssd
import imageio